You are given a huge logfiles of NUCES students accessing google classroom. We want to analyze the file and extract some basic statistics and useful patterns from it.

The format of the input file is as follows:

**``RollNo Login time Logout time list of pages accessed during this time``**

Example Input file

L20-4305 Course: BigData Sem: Spring2020 Login:12-03-20-12:45 Logout:12-03-20-2:45 Accessed: stream, assignment
L20-1111 Course:DataMining Sem:Spring2020 Login:12-03-20-11:00 Logout:12-03-20-12:00 Accessed: quiz, material, assignment
L20-4305 Course:DataMining Sem:Spring2020 Login:12-03-20-12:00 Logout:12-03-20-2:00 Accessed: quiz, material, assignment
L20-1111 Course:DataMining Sem:Spring2020 Login:12-03-20-2:00 Logout:12-03-20-3:00 Accessed: quiz, material, assignment
L20-4305 Course:BigData Sem:Spring2020 Login:12-03-20-12:00 Logout:12-03-20-1:00 Accessed: quiz, material, assignment

In [1]:
# write to text file
# format of text file
# RollNo Login time Logout time list of pages accessed during this time
"""L20-4305 Course: BigData Sem: Spring2020 Login:12-03-20-12:45 Logout:12-03-20-2:45 Accessed: stream, assignment
L20-1111 Course:DataMining Sem:Spring2020 Login:12-03-20-11:00 Logout:12-03-20-12:00 Accessed: quiz, material, assignment
L20-4305 Course:DataMining Sem:Spring2020 Login:12-03-20-12:00 Logout:12-03-20-2:00 Accessed: quiz, material, assignment
L20-1111 Course:DataMining Sem:Spring2020 Login:12-03-20-2:00 Logout:12-03-20-3:00 Accessed: quiz, material, assignment
L20-4305 Course:BigData Sem:Spring2020 Login:12-03-20-12:00 Logout:12-03-20-1:00 Accessed: quiz, material, assignment
"""
rows = [ 'L20-4305 Course:BigData Sem:Spring2020 Login:12-03-20-12:45 Logout:12-03-20-2:45 Accessed: stream, assignment',
'L20-1111 Course:DataMining Sem:Spring2020 Login:12-03-20-11:00 Logout:12-03-20-12:00 Accessed: quiz, material, assignment',
'L20-4305 Course:DataMining Sem:Spring2020 Login:12-03-20-12:00 Logout:12-03-20-2:00 Accessed: quiz, material, assignment',
'L20-1111 Course:DataMining Sem:Spring2020 Login:12-03-20-2:00 Logout:12-03-20-3:00 Accessed: quiz, material, assignment',
'L20-4305 Course:BigData Sem:Spring2020 Login:12-03-20-12:00 Logout:12-03-20-1:00 Accessed: quiz, material, assignment']
with open('log.txt', 'w') as f:
    for row in rows:
        f.write(row + '\n')
        
        
import re
reg_roll = re.compile(r'L20-\d{4}')
reg_course = re.compile(r'Course:\w+')
reg_sem = re.compile(r'Sem:\w+')
reg_login = re.compile(r'Login:\d{2}-\d{2}-\d{2}-\d{1,2}:\d{2}')
reg_logout = re.compile(r'Logout:\d{2}-\d{2}-\d{2}-\d{1,2}:\d{2}')
reg_accessed = re.compile(r'Accessed:.*')
# add all regex to a list
# parsing this in python will be
import datetime
with open('log.txt', 'r') as f:
    for line in f:
        login = reg_login.search(line).group()
        logout = reg_logout.search(line).group()
        # access time only
        login = login.split('-')[-1]
        logout = logout.split('-')[-1]
        # convert to 24 hour format
        login = datetime.datetime.strptime(login, '%H:%M')
        logout = datetime.datetime.strptime(logout, '%H:%M')
        print(login, logout)    

1900-01-01 12:45:00 1900-01-01 02:45:00
1900-01-01 11:00:00 1900-01-01 12:00:00
1900-01-01 12:00:00 1900-01-01 02:00:00
1900-01-01 02:00:00 1900-01-01 03:00:00
1900-01-01 12:00:00 1900-01-01 01:00:00


In [2]:
%%writefile q1.py
# Question 1
# Write a Map Reduce algorithm to find the number of times a student accessed his each class on google 
# classroom during year 2020. You have to provide the pseudo-code for Mapper, Reducer and Combiner.
# You can use associative memory (array) in Mapper to make  your program efficient.
from mrjob.job import MRJob
import re

# regex for roll number
reg_roll = re.compile(r'L20-\d{4}')
# regex for class
reg_class = re.compile(r'Course:\w+')
# regex for accessed
reg_accessed = re.compile(r'Accessed:.*')

class timesaccessed(MRJob):
    def mapper(self, _, line):
        student = reg_roll.findall(line)[0]
        course = reg_class.findall(line)[0].split(':')[1]
        
        yield (student, course), 1
            
    def combiner(self, key, values):
        yield (key, sum(values))

    def reducer(self, key, values):
        yield (key, sum(values))

if __name__ == '__main__':
    timesaccessed.run()

Writing q1.py


In [3]:
! python q1.py log.txt

["L20-1111","DataMining"]	2
["L20-4305","BigData"]	2
["L20-4305","DataMining"]	1


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ammar\AppData\Local\Temp\q1.ammar.20240309.133330.167870
Running step 1 of 1...
job output is in C:\Users\ammar\AppData\Local\Temp\q1.ammar.20240309.133330.167870\output
Streaming final output from C:\Users\ammar\AppData\Local\Temp\q1.ammar.20240309.133330.167870\output...
Removing temp directory C:\Users\ammar\AppData\Local\Temp\q1.ammar.20240309.133330.167870...


In [4]:
%%writefile q2.py
# Question 2: For each course output the number of distinct students who have accessed the course
# classroom Write an efficient MapReduce algorithm to perform above task.

# Output:
# BigData 1
# DataMining 2

from mrjob.job import MRJob
import re

# regex for roll number
reg_roll = re.compile(r'L20-\d{4}')
# regex for class
reg_class = re.compile(r'Course:\w+')

class distinctstudents(MRJob):
    def mapper(self, _, line):
        student = reg_roll.findall(line)[0]
        course = reg_class.findall(line)[0].split(':')[1]
        
        yield course, student
            
    def reducer(self, key, values):
        # values are incoming students, get the distinct count
        yield key, len(set(values))
        
if __name__ == '__main__':
    distinctstudents.run()

Writing q2.py


In [5]:
! python q2.py log.txt

"BigData"	1
"DataMining"	2


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ammar\AppData\Local\Temp\q2.ammar.20240309.133330.916094
Running step 1 of 1...
job output is in C:\Users\ammar\AppData\Local\Temp\q2.ammar.20240309.133330.916094\output
Streaming final output from C:\Users\ammar\AppData\Local\Temp\q2.ammar.20240309.133330.916094\output...
Removing temp directory C:\Users\ammar\AppData\Local\Temp\q2.ammar.20240309.133330.916094...


In [141]:
%%writefile q3.py
# Question 3:
# For each student find the percentage of time spent by the student in a course in Spring 2020. Write an efficient MapReduce
# algorithm to perform above task.
# Percentage of the time spend by a student A in course B = (Time spend by the student A in a course B classroom / Total time spent by all students in course B classroom) * 100

# Output format for above Question
# (Student rollnumber, course -> Percentage of the time spend by a student A in course B (sem C))
# Output for given input
# L20-4305, BigData -> 100%
# L20-4305, DataMining -> 60%
# L20-1111, DataMining -> 40%

# Hint: This problem is similar to relative frequency word co-occurrence problem discussed in the class.


import re
from mrjob.job import MRJob

reg_login = re.compile(r'Login:\d{2}-\d{2}-\d{2}-\d{1,2}:\d{2}')
reg_logout = re.compile(r'Logout:\d{2}-\d{2}-\d{2}-\d{1,2}:\d{2}')

def convert_to_seconds(time_str):
    hours, minutes = map(int, time_str.split(':'))
    seconds = hours * 3600 + minutes * 60
    return seconds

class CourseAccess(MRJob):

    def mapper(self, _, line):
        # Extract relevant information from the record
        student_rollnumber, course = line.split(' ')[0], line.split(' ')[1]
        login = reg_login.search(line).group()
        logout = reg_logout.search(line).group()
        # access time only
        login = login.split('-')[-1]
        logout = logout.split('-')[-1]
        # convert to 24-hour format
        login_seconds = convert_to_seconds(login)
        logout_seconds = convert_to_seconds(logout)
        if logout_seconds < login_seconds:
            logout_seconds += 86400  # add 24 hours
        time_spent = logout_seconds - login_seconds
        # Emit intermediate key-value pair
        yield (student_rollnumber, course), time_spent
                
    def reducer(self, key, course_time_spent):
        student, course = key
        # course_time = 0
        # for course, _ in key:
        #     course_time += course_time_spent
        total_time_spent = sum(course_time_spent)
        yield (student, course), total_time_spent
        
if __name__ == '__main__':
    CourseAccess.run()


Overwriting q3.py


In [142]:
! python q3.py log.txt

["L20-1111","Course:DataMining"]	7200
["L20-4305","Course:BigData"]	97200
["L20-4305","Course:DataMining"]	50400


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ammar\AppData\Local\Temp\q3.ammar.20240309.150405.162718
Running step 1 of 1...
job output is in C:\Users\ammar\AppData\Local\Temp\q3.ammar.20240309.150405.162718\output
Streaming final output from C:\Users\ammar\AppData\Local\Temp\q3.ammar.20240309.150405.162718\output...
Removing temp directory C:\Users\ammar\AppData\Local\Temp\q3.ammar.20240309.150405.162718...
